In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

import xopr.opr_access as xopr

In [ ]:
# Establish an OPR session
# The purpose of this right now is just to define a local cache directory, but 
# in the future we might use it to determine which STAC catalog to use or something.
opr = xopr.OPRConnection(cache_dir="radar_cache")

In [ ]:
# Use xopr to query the STAC catalog for radar frames within a bounding box

# TODO: Placeholder. No STAC catalog yet.

In [ ]:
# Placeholder for loading data

#urls = [f"https://data.cresis.ku.edu/data/rds/2022_Antarctica_BaslerMKB/CSARP_standard/20221210_01/Data_20221210_01_{i:03d}.mat" for i in range(1, 10)]
urls = [f'https://data.cresis.ku.edu/data/rds/2016_Antarctica_DC8/CSARP_standard/20161026_05/Data_20161026_05_{i:03d}.mat' for i in range(1, 10)]

frames = [opr.load_frame(url) for url in urls]

In [ ]:
# Inspect an individual frame
frames[0]

In [ ]:
# Combine the frames into a single xarray Dataset representing the flight line
flight_line = xr.concat(frames, dim='slow_time', combine_attrs='drop_conflicts')
flight_line = xopr.get_layers(flight_line)
flight_line

In [ ]:
# Plot a map of where the data was collected with a basemap on an EPSG:3031 projection
proj = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
coords = flight_line[['Longitude', 'Latitude']].to_dataframe().dropna()
(gf.ocean * gf.coastline * gv.Path([coords.values])).opts(projection=proj, aspect='equal')


In [ ]:
stacked = flight_line.Data.rolling(slow_time=10, center=True).mean()
pwr_dB = 10*np.log10(np.abs(stacked))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
pwr_dB.plot.imshow(x='slow_time', cmap='gray', ax=ax)
ax.invert_yaxis()

flight_line['layer_1_twtt'].plot(ax=ax, x='slow_time')

In [ ]:
print(xopr.generate_citation(flight_line))